# Plastic Classifier

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/surfriderfoundationeurope/IA_Pau/blob/master/Hackaton_Surfrider_Getting_Started.ipynb)


The goal is to build a plastic classifier, as the core detector / tracker is already built (but only works for generic plastic).

If you want fast training, make sure you have a good GPU: check using the command `!nvidia-smi`

Please install the required packages:   
`pip install -r requirements.txt`

### Getting the data

To get the images of [taco](http://tacodataset.org/) project, you download the dataset from [here](https://www.kaggle.com/kneroma/tacotrashdataset).        
Then put the `data` content folder in the `data/images/` folder of this repository, to have this structure:
- `data/images/`
    - `bach_1/`
    - `bach_2/`
    - `bach_3/`
    - ...
    - `annotations.json`

## Analyse the dataset

the next following cells enable you to get a bit of information about the dataset

In [1]:
import sys
sys.path.insert(0,'../')

In [ ]:
import json 
from pycocotools.coco import COCO
import matplotlib.pyplot as plt


coco = COCO(annotation_file = './data/images/annotations/instances_train.json')

coco_categories = coco.dataset['categories'][1:]

nb_anns_per_cat = {cat['name']: len(coco.getAnnIds(catIds=[cat['id']])) for cat in coco_categories}
nb_anns_per_cat = {k:v for k,v in sorted(nb_anns_per_cat.items(), key=lambda x: x[1], reverse=True)}
cat_names = list(nb_anns_per_cat.keys())
nb_images = list(nb_anns_per_cat.values())

plt.bar(x = cat_names, height = nb_images)
plt.xticks(range(len(cat_names)), cat_names, rotation='vertical')
plt.ylabel('Number of annotations')
plt.tight_layout()
plt.autoscale(True)
plt.savefig('dataset_analysis')

In [ ]:
import os
from detection.coco_utils import CocoDetectionWithExif, ConvertCocoPolysToBboxes

def get_dataset(root, image_set):
    PATHS = {
        "train": ("images", os.path.join("annotations", "instances_train.json")),
        "val": ("images", os.path.join("annotations", "instances_val.json")),
    }

    img_folder, ann_file = PATHS[image_set]
    img_folder = os.path.join(root, img_folder)
    ann_file = os.path.join(root, ann_file)

    dataset = CocoDetectionWithExif(img_folder, ann_file, transforms=ConvertCocoPolysToBboxes())

    return dataset

In [ ]:
from detection.coco_utils import get_surfrider
from detection import transforms

base_size = 540
crop_size = (544, 960)
downsampling_factor = 4
num_classes = 10
path = '/content/surfnet/data/images/'

# Building a train & test dataset
train_dataset = get_dataset(path, "train")
val_dataset = get_dataset(path, "val")

Let us display a full size picture, and corresponding bounding box

In [ ]:
import matplotlib.pyplot as plt
x, y = next(iter(train_dataset))
print(x.shape, y)
plt.imshow(x)